In [ ]:
from lm_polygraph.utils import UEManager

In [ ]:
man_cal = UEManager.load('./resources/llm_resources/llama8b_xsum.man') # просто чтобы напечатать, какие меры неопределённости есть
for el in man_cal.estimations.keys():
    print(el)

In [ ]:
from mdu.unc.entropic_ot import EntropicOTOrdering
from mdu.unc.constants import OTTarget, SamplingMethod, ScalingType
import numpy as np

In [ ]:
# это можно не менять
hyperparams_list = [
    {
        'target': OTTarget.EXP,
        'sampling_method': SamplingMethod.GRID,
        'scaling_type': ScalingType.FEATURE_WISE,
        'grid_size': 5,
        'n_targets_multiplier': 1,
        'eps': 0.5,
        'max_iters': 1000,
        'tol': 1e-6,
    },
    {
        'target': OTTarget.EXP,
        'sampling_method': SamplingMethod.GRID,
        'scaling_type': ScalingType.GLOBAL,
        'grid_size': 5,
        'n_targets_multiplier': 1,
        'eps': 0.5,
        'max_iters': 1000,
        'tol': 1e-6,
    },
    {
        'target': OTTarget.BETA,
        'sampling_method': SamplingMethod.GRID,
        'scaling_type': ScalingType.FEATURE_WISE,
        'grid_size': 5,
        'n_targets_multiplier': 1,
        'eps': 0.5,
        'max_iters': 1000,
        'tol': 1e-6,
    },
    {
        'target': OTTarget.BETA,
        'sampling_method': SamplingMethod.GRID,
        'scaling_type': ScalingType.GLOBAL,
        'grid_size': 5,
        'n_targets_multiplier': 1,
        'eps': 0.5,
        'max_iters': 1000,
        'tol': 1e-6,
    },
]

In [ ]:
calib_ratio = 0.25 # это доля объектов, которая будет использована для калибровки (обучения ОТ). Её можно менять
import pickle

for model_name in ["llama8b", "falcon7b", "mistral7b"]:

    file_names = [
        f'{model_name}_xsum.man',
        f'{model_name}_wmt14_fren.man',
        f'{model_name}_wmt19_deen.man',
        f'{model_name}_trivia.man',
        f'{model_name}_mmlu.man',
        f'{model_name}_gsm8k_cot.man',
        f'{model_name}_coqa_no_context.man',
    ]

    for file_name in file_names:
        man_current = UEManager.load(file_name)

        measures_dict = {k: v for k, v in man_current.estimations.items()}

        # Это я вручную создаю CoCoA меры (я проглядел, что они уже были в исходном менеджере, но под другими именами)
        measures_dict[('sequence', 'CoCoA_greedy_MSP')] = man_current.estimations[('sequence', 'GreedyAveDissimilarity')] * man_current.estimations[('sequence', 'MaximumSequenceProbability')]
        measures_dict[('sequence', 'CoCoA_greedy_PPL')] = man_current.estimations[('sequence', 'GreedyAveDissimilarity')] * man_current.estimations[('sequence', 'Perplexity')]
        measures_dict[('sequence', 'CoCoA_greedy_MTE')] = man_current.estimations[('sequence', 'GreedyAveDissimilarity')] * man_current.estimations[('sequence', 'MeanTokenEntropy')]

        measures_dict[('sequence', 'CoCoA_best_MSP')] = man_current.estimations[('sequence', 'BestAveDissimilarity')] * man_current.estimations[('sequence', 'BestSampledMaximumSequenceProbability')]
        measures_dict[('sequence', 'CoCoA_best_PPL')] = man_current.estimations[('sequence', 'BestAveDissimilarity')] * man_current.estimations[('sequence', 'BestSampledPerplexity')]
        measures_dict[('sequence', 'CoCoA_best_MTE')] = man_current.estimations[('sequence', 'BestAveDissimilarity')] * man_current.estimations[('sequence', 'BestSampledMeanTokenEntropy')]

        n_samples = len(man_current.estimations[('sequence', 'Perplexity')])
        np.random.seed(42)
        # Это я фиксирую калибровочную маску
        calib_mask = np.random.choice(n_samples, size=int(n_samples * calib_ratio), replace=False)

        final_measures = {}

        final_measures['calibration_mask'] = calib_mask

        for hyperparams in hyperparams_list:
            target = hyperparams['target']
            sampling_method = hyperparams['sampling_method']
            scaling_type = hyperparams['scaling_type']
            grid_size = hyperparams['grid_size']
            n_targets_multiplier = hyperparams['n_targets_multiplier']
            eps = hyperparams['eps']
            max_iters = hyperparams['max_iters']
            tol = hyperparams['tol']


            model = EntropicOTOrdering(
                target=target,
                sampling_method=sampling_method,
                scaling_type=scaling_type,
                grid_size=grid_size,
                target_params={},
                eps=eps,
                n_targets_multiplier=n_targets_multiplier,
                max_iters=max_iters,
                random_state=42,
                tol=tol,
            )

            combinations = [
                (
                    f"CoCoA_like_1_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'GreedyAveDissimilarity'),
                        ('sequence', 'BestAveDissimilarity'),
                        ('sequence', 'BestSampledMaximumSequenceProbability'),
                        ('sequence', 'BestSampledPerplexity'),
                        ('sequence', 'BestSampledMeanTokenEntropy'),
                        ('sequence', 'MaximumSequenceProbability'),
                        ('sequence', 'Perplexity'),
                        ('sequence', 'MeanTokenEntropy'),
                    )
                ),
                (
                    f"CoCoA_like_2_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'BestAveDissimilarity'),
                        ('sequence', 'BestSampledMaximumSequenceProbability'),
                        ('sequence', 'BestSampledPerplexity'),
                        ('sequence', 'BestSampledMeanTokenEntropy'),
                    )
                ),
                (
                    f"CoCoA_like_3_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'GreedyAveDissimilarity'),
                        ('sequence', 'MaximumSequenceProbability'),
                        ('sequence', 'Perplexity'),
                        ('sequence', 'MeanTokenEntropy'),
                    )
                ),
                (
                    f"Master_1_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_greedy_MSP'),
                        ('sequence', 'CoCoA_greedy_PPL'),
                        ('sequence', 'CoCoA_greedy_MTE'),
                    )
                ),
                (
                    f"Master_1_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_greedy_MSP'),
                        ('sequence', 'CoCoA_greedy_PPL'),
                        ('sequence', 'CoCoA_greedy_MTE'),
                    )
                ),
                (
                    f"Master_2_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_best_MSP'),
                        ('sequence', 'CoCoA_best_PPL'),
                        ('sequence', 'CoCoA_best_MTE'),
                    )
                ),
                (
                    f"Master_3_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_best_MSP'),
                        ('sequence', 'CoCoA_best_PPL'),
                        ('sequence', 'CoCoA_best_MTE'),
                        ('sequence', 'CoCoA_greedy_MSP'),
                        ('sequence', 'CoCoA_greedy_PPL'),
                        ('sequence', 'CoCoA_greedy_MTE'),
                    )
                ),
                (
                    f"IMBA_1_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_best_MSP'),
                        ('sequence', 'CoCoA_best_PPL'),
                        ('sequence', 'DegMat_NLI_score_entail'),
                    )
                ),
                (
                    f"IMBA_2_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_greedy_MSP'),
                        ('sequence', 'CoCoA_greedy_PPL'),
                        ('sequence', 'CoCoA_greedy_MTE'),
                        ('sequence', 'DegMat_NLI_score_entail'),  # это имя, под которым будет сохраняться композитная мера
                    )
                ),
                (
                    f"IMBA_3_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_best_MSP'),
                        ('sequence', 'CoCoA_best_PPL'),
                        ('sequence', 'CoCoA_best_MTE'),
                        ('sequence', 'CoCoA_greedy_MSP'),
                        ('sequence', 'CoCoA_greedy_PPL'),
                        ('sequence', 'CoCoA_greedy_MTE'),
                        ('sequence', 'DegMat_NLI_score_entail'),
                        ('sequence', 'EigValLaplacian_NLI_score_entail'),
                    )
                ),
                (
                    f"IMBA_4_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_greedy_MSP'),
                        ('sequence', 'CoCoA_greedy_PPL'),
                        ('sequence', 'CoCoA_greedy_MTE'),
                        ('sequence', 'DegMat_NLI_score_entail'),
                        ('sequence', 'MaximumSequenceProbability'),
                    )
                ),
                (
                    f"IMBA_5_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_greedy_MSP'),
                        ('sequence', 'CoCoA_greedy_PPL'),
                        ('sequence', 'CoCoA_greedy_MTE'),
                        ('sequence', 'MaximumSequenceProbability'),
                    )
                ),
            ]

            for comb_list in combinations:
                name, comb = comb_list[0], comb_list[1]
                components = [measures_dict[el] for el in comb]
                
                calib_components = [np.array(comp)[calib_mask] for comp in components]

                stacked_calib_components = np.column_stack(calib_components)
                stacked_test_components = np.column_stack(components)

                model.fit(stacked_calib_components)
                scores = model.predict(stacked_test_components)

                final_measures[name] = scores

        with open(f'./resources/llm_resources/final_measures_{file_name.split(".")[0]}.pkl', 'wb') as f:
            pickle.dump(final_measures, f)